In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv


In [68]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [69]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [70]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [71]:
x = train.isnull().sum()

for i in range(len(x)):
  #  print(x[i])
    if(x[i]>0):
        print(train.columns[i]," : ",x[i])

LotFrontage  :  259
Alley  :  1369
MasVnrType  :  8
MasVnrArea  :  8
BsmtQual  :  37
BsmtCond  :  37
BsmtExposure  :  38
BsmtFinType1  :  37
BsmtFinType2  :  38
Electrical  :  1
FireplaceQu  :  690
GarageType  :  81
GarageYrBlt  :  81
GarageFinish  :  81
GarageQual  :  81
GarageCond  :  81
PoolQC  :  1453
Fence  :  1179
MiscFeature  :  1406


In [72]:
train.shape

(1460, 81)

In [73]:
train = train.drop(columns = ['Alley','FireplaceQu','PoolQC','Fence','MiscFeature'],axis = 1)
test = test.drop(columns = ['Alley','FireplaceQu','PoolQC','Fence','MiscFeature'],axis = 1)

In [74]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [75]:
empty = []
x = train.isnull().sum()

for i in range(len(x)):
  #  print(x[i])
    if(x[i]>0):
        print(train.columns[i]," : ",x[i])
        empty.append(train.columns[i])

LotFrontage  :  259
MasVnrType  :  8
MasVnrArea  :  8
BsmtQual  :  37
BsmtCond  :  37
BsmtExposure  :  38
BsmtFinType1  :  37
BsmtFinType2  :  38
Electrical  :  1
GarageType  :  81
GarageYrBlt  :  81
GarageFinish  :  81
GarageQual  :  81
GarageCond  :  81


In [76]:
'''
LotFrontage : mean
MasVnrType : fill 'None'
MasVnrArea : 0
BsmtQual : TA
BsmtCond : TA
BsmtExposure : No
BsmtFinType1 : Unf
BsmtFinType2 : Unf
Electrical : SBrkr
GarageType : Attchd
GarageYrBlt : 2005.0
GarageFinish : Unf
GarageQual : TA
GarageCond : TA
'''

"\nLotFrontage : mean\nMasVnrType : fill 'None'\nMasVnrArea : 0\nBsmtQual : TA\nBsmtCond : TA\nBsmtExposure : No\nBsmtFinType1 : Unf\nBsmtFinType2 : Unf\nElectrical : SBrkr\nGarageType : Attchd\nGarageYrBlt : 2005.0\nGarageFinish : Unf\nGarageQual : TA\nGarageCond : TA\n"

In [77]:
#Fill Empty columns
m = train['LotFrontage'].mean()
train['LotFrontage'] = train['LotFrontage'].fillna(m)
train['MasVnrType'] = train['MasVnrType'].fillna('None')
train['MasVnrArea'] = train['MasVnrArea'].fillna(0)
train['BsmtQual'] = train['BsmtQual'].fillna('TA')
train['BsmtCond'] = train['BsmtCond'].fillna('TA')
train['BsmtExposure'] = train['BsmtExposure'].fillna('No')
train['BsmtFinType1'] = train['BsmtFinType1'].fillna('Unf')
train['BsmtFinType2'] = train['BsmtFinType2'].fillna('Unf')
train['Electrical'] = train['Electrical'].fillna('SBrkr')
train['GarageType'] = train['GarageType'].fillna('Attchd')
train['GarageYrBlt'] = train['GarageYrBlt'].fillna(2005.0)
train['GarageFinish'] = train['GarageFinish'].fillna('Unf')
train['GarageQual'] = train['GarageQual'].fillna('TA')
train['GarageCond'] = train['GarageCond'].fillna('TA')

In [78]:
empty = []
x = train.isnull().sum()

for i in range(len(x)):
  #  print(x[i])
    if(x[i]>0):
        print(train.columns[i]," : ",x[i])
        empty.append(train.columns[i])
print(empty)

[]


# Now that all the entries are filled we can convert them into suitable data-type

In [79]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [80]:
obj_new_list=['MSSubClass','OverallQual','OverallCond','BsmtFullBath','GarageCars','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd'
              ,'Fireplaces','MoSold']
train[obj_new_list]

,MSSubClass,OverallQual,OverallCond,BsmtFullBath,GarageCars,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,MoSold
0,60,7,5,1,2,0,2,1,3,1,8,0,2
1,20,6,8,0,2,1,2,0,3,1,6,1,5
2,60,7,5,1,2,0,2,1,3,1,6,1,9
3,70,7,5,1,3,0,1,0,3,1,7,1,2
4,60,8,5,1,3,0,2,1,4,1,9,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,6,5,0,2,0,2,1,3,1,7,1,8
1456,20,6,6,1,2,0,2,0,3,1,7,2,2
1457,70,7,9,0,1,0,2,0,4,1,9,2,5
1458,20,5,6,1,1,0,1,0,2,1,5,0,4


I have divided sets into 3 parts -

    1. Object type variables 
    2. Integer type variables -- The missing values would be added with **median** values
    3. Integer turned Category variables 

In [81]:
## Convert to category variable

#Object
obj_typ_var = [column for column in train.columns if train[column].dtype in ['object']]
obj_data = train[obj_typ_var]
obj_test_data = test[obj_typ_var]

obj_new_list=['MSSubClass','OverallQual','OverallCond','BsmtFullBath','GarageCars','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd'
              ,'Fireplaces','MoSold']
## abs          Convert all the data in obj_new_ist in ''OBJECT'' format
train[obj_new_list] = train[obj_new_list].apply(lambda column:column.astype('object'))
test[obj_new_list] = test[obj_new_list].apply(lambda column:column.astype('object'))

category_data = train[obj_new_list]
category_data_test = test[obj_new_list]

##Int
int_type_var = [column for column in train.columns if train[column].dtype in ['int64','float64']]
int_data = train[int_type_var]
int_data_test = test[int_type_var[:-1]]


In [82]:
print(int_type_var[:-1])

['Id', 'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'YrSold']


In [83]:
obj_test_data['BsmtQual']=obj_test_data['BsmtQual'].fillna("TA")
obj_test_data['BsmtCond']=obj_test_data['BsmtCond'].fillna("TA")
obj_test_data['BsmtExposure']=obj_test_data['BsmtExposure'].fillna("No")
obj_test_data['BsmtFinType1']=obj_test_data['BsmtFinType1'].fillna("Unf")
obj_test_data['BsmtFinType2']=obj_test_data['BsmtFinType2'].fillna("Unf")
obj_test_data['Electrical']=obj_test_data['Electrical'].fillna("SBrkr")
obj_test_data['GarageType']=obj_test_data['GarageType'].fillna("Attchd")
obj_test_data['GarageFinish']=obj_test_data['GarageFinish'].fillna("Unf")
obj_test_data['GarageQual']=obj_test_data['GarageQual'].fillna("TA")
obj_test_data['GarageCond']=obj_test_data['GarageCond'].fillna("TA")
obj_test_data['PoolQC']=obj_test_data['GarageCond'].fillna("TA")

test['BsmtHalfBath']=test['BsmtHalfBath'].fillna("0")
test['BsmtFullBath']=test['BsmtFullBath'].fillna("0")
test['GarageCars']=test['GarageCars'].fillna("2")


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [84]:
obj_test_data.isna().sum()

MSZoning          4
Street            0
LotShape          0
LandContour       0
Utilities         2
LotConfig         0
LandSlope         0
Neighborhood      0
Condition1        0
Condition2        0
BldgType          0
HouseStyle        0
RoofStyle         0
RoofMatl          0
Exterior1st       1
Exterior2nd       1
MasVnrType       16
ExterQual         0
ExterCond         0
Foundation        0
BsmtQual          0
BsmtCond          0
BsmtExposure      0
BsmtFinType1      0
BsmtFinType2      0
Heating           0
HeatingQC         0
CentralAir        0
Electrical        0
KitchenQual       1
Functional        2
GarageType        0
GarageFinish      0
GarageQual        0
GarageCond        0
PavedDrive        0
SaleType          1
SaleCondition     0
PoolQC            0
dtype: int64

# Build Model

In [85]:
obj_typ_var = list(set(obj_typ_var + obj_new_list))

train_X_obj = train[obj_typ_var].apply(lambda column : column.astype('category'))
test_X_obj = test[obj_typ_var].apply(lambda column : column.astype('category'))

In [86]:
combined_data = pd.concat((train[obj_typ_var], test[obj_typ_var]), sort=False).reset_index(drop=True)
combined_data_dummies=pd.get_dummies(combined_data)

In [87]:
train_obj = combined_data_dummies.iloc[:train[obj_typ_var].shape[0]].reset_index(drop=True)
test_obj = combined_data_dummies.iloc[train[obj_typ_var].shape[0]:].reset_index(drop=True)

In [88]:
int_selected = int_type_var[1:-1]
int_selected

['LotFrontage',
 'LotArea',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageYrBlt',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'YrSold']

In [89]:
train_int = train[int_selected]
test_int = test[int_selected]

In [90]:
train_int = train_int.fillna(train_int.median())
test_int = train_int.fillna(test_int.median())

In [91]:
from sklearn.preprocessing import MinMaxScaler

In [92]:
sclr = MinMaxScaler()
train_int = pd.DataFrame(sclr.fit_transform(train_int),columns = train_int.columns)
test_int = pd.DataFrame(sclr.transform(test_int),columns = test_int.columns)

In [93]:
train_Y = train['SalePrice']

In [94]:
train_X = train_int.merge(train_obj,left_index=True,right_index=True).reset_index(drop=True)
test_X = test_int.merge(test_obj,left_index=True,right_index=True).reset_index(drop=True)

In [95]:
test_X.columns[test_X.isna().any()].tolist()

[]

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_Y,test_size = .3, random_state=0)

In [97]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [98]:
rf = RandomForestRegressor()
params = {"max_depth":[15,20,25], "n_estimators":[24,30,36]}
rf_reg = GridSearchCV(rf, params, cv = 10, n_jobs =10)
rf_reg.fit(X_train, Y_train)
print(rf_reg.best_estimator_)
best_estimator=rf_reg.best_estimator_
y_pred_test = best_estimator.predict(X_test)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=30, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)


In [99]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [100]:
from sklearn.ensemble import GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor(n_estimators=1992, learning_rate=0.03, max_depth=3, max_features='sqrt', min_samples_leaf=15, min_samples_split=8, loss='huber', random_state =42)
gb_regressor.fit(X_train,Y_train)
y_pred_test=gb_regressor.predict(X_test)


In [101]:
print('Mean Square Error test = ' + str(mean_squared_error(Y_test, y_pred_test)))
print('Root Mean Square Error test = ' + str(np.sqrt(mean_squared_error(Y_test, y_pred_test))))

print ('R2 square '+str(r2_score(Y_test, y_pred_test))) 

Mean Square Error test = 918257397.5056148
Root Mean Square Error test = 30302.76220917187
R2 square 0.8647431793822747


In [102]:
pred_test = gb_regressor.predict(test_X)

In [103]:
sample_submission=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
submission=pd.DataFrame({"Id":sample_submission['Id'],"SalePrice":pred_test})
submission.to_csv('submission.csv',index=False)

In [104]:
sample_submission.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
